In [23]:
# test classification dataset
from sklearn.datasets import make_classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score, recall_score
import RIfunctions as ri
from imblearn.under_sampling import NearMiss
from sklearn.model_selection import GridSearchCV
import joblib

In [2]:
data = pd.read_csv('input_file_2.csv', sep = ',', index_col=0)

In [3]:
data['issue_d'] = pd.to_datetime(data['issue_d'])

In [4]:
data = data.loc[data['issue_d'] < '2016-1-1']

In [5]:
data = pd.get_dummies(data,prefix=["purp"], columns=['purpose'], drop_first=False)

In [6]:
train_df = data.loc[data['issue_d'] < data['issue_d'].quantile(0.80)]
test_df = data.loc[data['issue_d'] >= data['issue_d'].quantile(0.80)]

In [7]:
train_df['emp_length'].fillna(train_df['emp_length'].mean(), inplace = True)
test_df['emp_length'].fillna(test_df['emp_length'].mean(), inplace=True)

c:\Users\joshy\anaconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [8]:
scaler = StandardScaler(copy=False)
numerical_cols = ['loan_amnt', 'emp_length', 'dti']

train_df[numerical_cols] = scaler.fit_transform(train_df[numerical_cols], train_df['charged_off'])
test_df[numerical_cols] = scaler.transform(test_df[numerical_cols])

print(scaler.mean_)

<ipython-input-8-65ee4bc3d695>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[numerical_cols] = scaler.fit_transform(train_df[numerical_cols], train_df['charged_off'])
c:\Users\joshy\anaconda3\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


[1.40368424e+04 5.95408279e+00 1.73190424e+01]


<ipython-input-8-65ee4bc3d695>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[numerical_cols] = scaler.transform(test_df[numerical_cols])
c:\Users\joshy\anaconda3\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


In [9]:
# # Downsampling. No logical strategy.

# print('train and test shape     ', train_df.shape, test_df.shape)

# train_df = ri.balanceData(train_df)
# test_df =  ri.balanceData(test_df)

# print('train and test shape     ', train_df.shape, test_df.shape)
# print('value counts train     ', train_df['charged_off'].value_counts())
# print('value counts test     ', test_df['charged_off'].value_counts())

In [10]:
all_cols = list(data.columns)

for i in all_cols:
    if i.startswith("purp"):
        numerical_cols.append(i)

print(numerical_cols) 

train_df_y = train_df['charged_off']
train_df_x = train_df[numerical_cols]

test_df_x = test_df[numerical_cols]
test_df_y = test_df['charged_off']

['loan_amnt', 'emp_length', 'dti', 'purp_car', 'purp_credit_card', 'purp_debt_consolidation', 'purp_educational', 'purp_home_improvement', 'purp_house', 'purp_major_purchase', 'purp_medical', 'purp_moving', 'purp_other', 'purp_renewable_energy', 'purp_small_business', 'purp_vacation', 'purp_wedding']


In [12]:
ns = NearMiss(sampling_strategy='not minority')

train_df_x, train_df_y  = ns.fit_resample(train_df_x, train_df_y)
print(pd.DataFrame(train_df_y.value_counts()))

test_df_x, test_df_y = ns.fit_resample(test_df_x, test_df_y)
print(pd.DataFrame(test_df_y.value_counts()))

   charged_off
0        91554
1        91554
   charged_off
0        39298
1        39298


In [13]:
# # Synthesise binary classification dataset
# X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=3)
# # summarize the dataset
print(train_df_x.shape, train_df_y.shape)

(183108, 17) (183108,)


In [14]:
# define the model
model = RandomForestClassifier(n_estimators=500, random_state=42)
# fit the model on the whole dataset
model.fit(train_df_x, train_df_y)

RandomForestClassifier(n_estimators=500, random_state=42)

In [ ]:
# # evaluate the model
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# n_scores = cross_val_score(model, train_df_x, train_df_y, scoring='accuracy', cv=cv, n_jobs=-1)
# # report performance
# print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [15]:
# 10 fold cross validation
rf_brforetune_cv_scores = cross_val_score(estimator= model, X=train_df_x, y=train_df_y, cv=10, n_jobs=-1)
mean_rf_brforetune_cv_score =rf_brforetune_cv_scores.mean()
print("The cross-validation accuracy score for untuned Random Forest model after a 10 fold cross validation:", mean_rf_brforetune_cv_score)
print('Accuracy: %.3f (%.3f)' % (mean(rf_brforetune_cv_scores), std(rf_brforetune_cv_scores)))

The cross-validation accuracy score for untuned Random Forest model after a 10 fold cross validation: 0.6111697892481565
Accuracy: 0.611 (0.076)


In [19]:
print(model.get_params())

params_rf ={
          'n_estimators': [100,300,400,500, 600,1000],
          'criterion': ["gini", "entropy"],
          'max_depth': [100,200, 300, 400, 1000],
          'max_features': ["log2", "sqrt"],
          'bootstrap':[True, False]
}

grid_rf = GridSearchCV(estimator=model, param_grid=params_rf ,cv = 3, scoring="neg_mean_squared_error", verbose = 1, n_jobs=-1)
grid_rf.fit(train_df_x, train_df_y)

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 500, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}
Fitting 3 folds for each of 240 candidates, totalling 720 fits


GridSearchCV(cv=3,
             estimator=RandomForestClassifier(n_estimators=500,
                                              random_state=42),
             n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [100, 200, 300, 400, 1000],
                         'max_features': ['log2', 'sqrt'],
                         'n_estimators': [100, 300, 400, 500, 600, 1000]},
             scoring='neg_mean_squared_error', verbose=1)

In [16]:
yhat = model.predict(test_df_x)

In [17]:
print(roc_auc_score(test_df_y, yhat), recall_score(test_df_y, yhat, pos_label=1), recall_score(test_df_y, yhat, pos_label=0))

0.6594483179805588 0.7947478243167592 0.5241488116443584


300 Trees: 0.6597664003257163 0.7947732709043718 0.524759529747061
500 Trees: 0.6594483179805588 0.7947478243167592 0.5241488116443584

In [20]:
best_hyperparams = grid_rf.best_params_
print("Best hyperparameters: \n", best_hyperparams )
best_rf = grid_rf.best_estimator_

Best hyperparameters: 
 {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 100, 'max_features': 'log2', 'n_estimators': 100}


In [22]:

best_rf.save('RF Optimised Hyperparamters')

AttributeError: 'RandomForestClassifier' object has no attribute 'save'

In [24]:
joblib.dump(best_rf, "./random_forest.joblib")

['./random_forest.joblib']